# Reprojecting Sentinel-1 data on GFW grid

In [ ]:
from osgeo import gdal, gdalconst
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# this allows GDAL to throw Python Exceptions
gdal.UseExceptions()

pathData = "/home/ec2-user/data/"

In [ ]:
# Read labels dataset
label_filename = pathData + 'GFWLabels2017.tiff'
labels_dataset = gdal.Open(label_filename)

labels_nx = labels_dataset.RasterXSize  # Raster xsize
labels_ny = labels_dataset.RasterYSize  # Raster ysize

labels_gt = labels_dataset.GetGeoTransform()
labels_proj = labels_dataset.GetProjection()
labels_xres = labels_gt[1]
labels_yres = labels_gt[5]

labels_lrx = labels_gt[0] + (labels_nx * labels_xres)
labels_lry = labels_gt[3] + (labels_ny * labels_yres)

### Mosaicking and reprojecting S1 data 

In [ ]:
#August Data
dst_filename = pathData + 'S1_Aug17_GFW_grid.tif'
src_filelist = [pathData + 'S1_GMS_Aug2017_extended_domain-0000000000-0000007424.tif',
                pathData + 'S1_GMS_Aug2017_extended_domain-0000000000-0000000000.tif']

gdal.Warp(dst_filename, src_filelist, dstSRS = labels_proj,
          xRes = labels_xres, yRes = labels_yres, outputBounds=[labels_gt[0], labels_lry, labels_lrx, labels_gt[3]],
          resampleAlg = gdalconst.GRA_Mode)

# Check reprojected data
dst_dataset = gdal.Open(dst_filename)

dst_nx = dst_dataset.RasterXSize
dst_ny = dst_dataset.RasterYSize
dst_gt = dst_dataset.GetGeoTransform()
dst_xres = dst_gt[1]
dst_yres = dst_gt[5]
dst_lrx = dst_gt[0] + (dst_nx * dst_xres)
dst_lry = dst_gt[3] + (dst_ny * dst_yres)

labels_lrx = labels_gt[0] + (labels_nx * labels_xres)
labels_lry = labels_gt[3] + (labels_ny * labels_yres)


In [ ]:
# Visualize Sample Data
from readGeotiff import readGeotiff

labels, nx, ny, xres, yres, gt, proj = readGeotiff(pathData + 'S1_Aug17_GFW_grid.tif')

fig = plt.figure(figsize=(50, 20))
ax = fig.add_subplot(2,1,1)
cs = ax.imshow(np.nanmean(labels_mode[700:850, 2300:2500, :], 2), clim=(-15, 0))

ax2 = fig.add_subplot(2,1,2)
cs2 = ax2.imshow(np.nanmean(labels[700:850, 2300:2500, :], 2), clim=(-15, 0))